## Module 10 Assignment - Scraping a Website Part - I
* Author: brandon chiazza
* version 2.0

We will be creating a web scraper to parse a table from the Charities Bureau Website. From the website: “All 
charitable organizations operating in New York State are required by law to register and file annual financial reports 
with the Attorney General's Office. This includes any organization that conducts charitable activities, holds property 
that is used for charitable purposes, or solicits financial or other contributions.”

# Create web-scraper to load csv file into S3 Bucket

In [1]:
!pip install webdriver-manager

In [2]:
!pip install awscli

In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [14]:
###Load modules
#!pip install webdriver-manager
#!pip install awscli
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait  # Import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [15]:
####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table


# Loop through pages
while True:
    # Scraping table data
    for row in table.find_elements(By.CSS_SELECTOR, 'tr'):
        cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')])

    # Check for the next page link and click if available
    try:
        page_link = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[9]'))
        )
        page_link.click()
        # Wait for the next page to load
        sleep(4)
        table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')
    except:
        # Break the loop if the next page link is not found
        break

# Update dataframe with header 
df = pd.DataFrame(df, columns=["Organization Name", "NY Reg #", "EIN", "Registrant Type", "City", "State"])

df.dropna(inplace=True)
df #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
6,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
7,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
8,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
9,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
10,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


In [30]:
import boto3
import pandas as pd

# Specifying the S3 bucket name
bucket_name = 'database-update-bucket-m10-assn-yudhishna-kuppala'

# Prepare CSV file name
filename = 'charities_bureau_scrape_'  # Name of your group
datetime = time.strftime("%Y%m%d%H%M%S")  # Timestamp
s3_key = f"{filename}{datetime}.csv"  # S3 key (filename)

# Convert DataFrame to CSV string format
csv_buffer = df.to_csv(index=False)

# Uploading CSV file to S3
s3 = boto3.resource('s3')
s3_object = s3.Object(bucket_name, s3_key)
s3_object.put(Body=csv_buffer)

# Print success message
print("Successfully uploaded file to location: s3://{}/{}".format(bucket_name, s3_key))


Successfully uploaded file to location: s3://database-update-bucket-m10-assn-yudhishna-kuppala/charities_bureau_scrape_20240413014815.csv


## References
* https://www.programiz.com/python-programming/working-csv-files
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
* https://realpython.com/python-boto3-aws-s3/
* https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 